In [ ]:
# Dataset loading
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load data
df = pd.read_csv("breast-cancer.csv")

# # Drop ID column
df.drop(columns=['id'], inplace=True)

# # Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2
)

# # Feature scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)

# # Label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test  = encoder.transform(y_test)

# # Convert to tensors
x_train_tensor = torch.from_numpy(x_train).float()
x_test_tensor  = torch.from_numpy(x_test).float()

y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
y_test_tensor  = torch.from_numpy(y_test).float().unsqueeze(1)

In [ ]:
class MySimpleNN:

    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, requires_grad=True)
        self.bias = torch.zeros(1, requires_grad=True)

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        return torch.sigmoid(z)

    def loss_function(self, y_pred, y):
        eps = 1e-7
        y_pred = torch.clamp(y_pred, eps, 1 - eps)
        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
model = MySimpleNN(x_train_tensor)

# Training loop
for epoch in range(epochs):

    # Forward pass
    y_pred = model.forward(x_train_tensor)

    # Loss calculation
    loss = model.loss_function(y_pred, y_train_tensor)
    # Backward pass
    loss.backward()

    # Update parameters
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias   -= learning_rate * model.bias.grad

    # Zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/25, Loss: 0.4215
Epoch 2/25, Loss: 0.4114
Epoch 3/25, Loss: 0.4010
Epoch 4/25, Loss: 0.3910
Epoch 5/25, Loss: 0.3813
Epoch 6/25, Loss: 0.3704
Epoch 7/25, Loss: 0.3606
Epoch 8/25, Loss: 0.3513
Epoch 9/25, Loss: 0.3416
Epoch 10/25, Loss: 0.3301
Epoch 11/25, Loss: 0.3199
Epoch 12/25, Loss: 0.3093
Epoch 13/25, Loss: 0.2997
Epoch 14/25, Loss: 0.2900
Epoch 15/25, Loss: 0.2807
Epoch 16/25, Loss: 0.2716
Epoch 17/25, Loss: 0.2628
Epoch 18/25, Loss: 0.2543
Epoch 19/25, Loss: 0.2461
Epoch 20/25, Loss: 0.2381
Epoch 21/25, Loss: 0.2304
Epoch 22/25, Loss: 0.2229
Epoch 23/25, Loss: 0.2158
Epoch 24/25, Loss: 0.2089
Epoch 25/25, Loss: 0.2023


In [ ]:
model.weights

tensor([[ 0.7165],
        [ 0.5831],
        [ 0.1484],
        [ 0.8762],
        [ 0.1491],
        [ 0.1324],
        [-0.0896],
        [ 0.8321],
        [-0.0856],
        [ 0.3915],
        [ 0.1982],
        [ 0.0758],
        [ 0.8620],
        [ 0.7634],
        [ 0.2795],
        [ 0.1639],
        [ 0.5489],
        [-0.1425],
        [ 0.2744],
        [-0.2405],
        [ 0.1772],
        [ 0.3458],
        [ 0.2779],
        [ 1.0201],
        [ 0.5283],
        [ 0.7812],
        [ 0.7006],
        [ 0.5773],
        [ 0.8124],
        [ 0.5832]], requires_grad=True)

In [ ]:
model.bias

tensor([-0.0597], requires_grad=True)

evaluation

In [ ]:
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy.item():.4f}")

# print(y_pred)

Accuracy: 0.9386
